# 🏦 Bank Customer Churn Prediction
## Notebook 3 — Data Cleaning

**Goal:** Produce a clean, analysis-ready DataFrame by:
1. Removing identifier columns (no predictive value).
2. Removing `Complain` (near-duplicate of the target → data leakage).
3. Documenting why we are NOT removing outliers in this dataset.
4. Saving the cleaned DataFrame as a checkpoint for N4.

> **What is data leakage?**  
> Leakage occurs when information from outside the valid training set leaks into the model.  
> `Complain` is only known *after* a customer has already churned — so using it would give the model "future" information it wouldn't have in a real deployment. The result: artificially high accuracy during training but **complete failure in production**.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='whitegrid')

data = pd.read_csv('Customer-Churn-Records.csv')
print(f'Original shape: {data.shape}')
data.head()

## 1. Confirm the `Complain` Data Leakage Problem

We documented this in EDA — let's formally quantify it here to justify the drop decision.

In [ ]:
corr_EC = data['Exited'].corr(data['Complain'])
identical_pct = (data['Exited'] == data['Complain']).mean() * 100

print(f'Pearson correlation  Exited ↔ Complain : {corr_EC:.4f}')
print(f'% rows where Exited == Complain         : {identical_pct:.1f}%')
print()
print('Breakdown (Exited , Complain):')
print(data.groupby(['Exited', 'Complain']).size().rename('Count').to_string())
print()
print('Verdict: Complain is essentially the same information as the target.')
print('         It MUST be dropped to prevent data leakage.')

# Visualise the near-perfect overlap
fig, ax = plt.subplots(figsize=(7, 4))
ct = pd.crosstab(data['Exited'], data['Complain'])
sns.heatmap(ct, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Complain=0', 'Complain=1'],
            yticklabels=['Exited=0', 'Exited=1'])
ax.set_title('Exited vs Complain Cross-Tabulation\n(near-perfect agreement confirms leakage)', fontsize=11)
plt.tight_layout()
plt.show()

## 2. Drop Columns

We drop four columns:

| Column | Reason |
|---|---|
| `RowNumber` | Sequential index — carries zero information |
| `CustomerId` | Unique ID — no generalizable pattern |
| `Surname` | Name — no generalizable pattern (and privacy concern) |
| `Complain` | Data leakage — near-duplicate of target |

The `axis=1` argument tells `drop()` to operate on columns (not rows).

In [ ]:
cols_to_drop = ['RowNumber', 'CustomerId', 'Surname', 'Complain']

df = data.drop(cols_to_drop, axis=1)

print(f'Shape before dropping : {data.shape}')
print(f'Shape after dropping  : {df.shape}')
print(f'Columns removed       : {cols_to_drop}')
print()
df.head()

## 3. Missing Values — Final Check

Although we confirmed no missing values in N2, it is good practice to re-verify after any column manipulation.

In [ ]:
missing = df.isnull().sum()
total_missing = missing.sum()

if total_missing == 0:
    print('✅ No missing values detected. Dataset is complete.')
else:
    print('⚠️ Missing values found:')
    print(missing[missing > 0])

## 4. Data Types — Final Check

We verify that all columns have the correct data type before passing the data to the feature engineering notebook.

In [ ]:
print('Column data types after cleaning:')
print(df.dtypes)
print()
print('Expected types:')
print('  object  → Geography, Gender, Card Type  (text, will be encoded in N4)')
print('  int64   → HasCrCard, IsActiveMember, Exited, ...  (numeric / binary)')
print('  float64 → Balance, EstimatedSalary  (continuous)')

## 5. Note on Outliers

A common step in data cleaning is **outlier removal**, but this dataset warrants no such action.  
Here is our reasoning:

- **`Balance = 0`** for ~36% of customers. This is a real and meaningful business state (customers who have no money in their account). Removing or imputing these values would distort the analysis.
- **`Age` range 18–92** — all plausible ages for bank customers.
- **`CreditScore` range 350–850** — valid credit score range.
- **`NumOfProducts` = 3 or 4** — unusual but valid; some customers hold many products.

**Rule of thumb:** Only remove outliers if they are *errors* (e.g., age = -5, salary = 999999999) or if they severely violate a model's assumptions. In tree-based models (our final choice), outliers have minimal impact because splits are based on rank, not magnitude.

In [ ]:
# Quick boxplots to visualise outlier extent
numerical_features = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'NumOfProducts']

fig, axes = plt.subplots(1, len(numerical_features), figsize=(18, 5))
for i, col in enumerate(numerical_features):
    axes[i].boxplot(df[col], patch_artist=True,
                    boxprops=dict(facecolor='#4C72B0', alpha=0.6))
    axes[i].set_title(col, fontsize=10)
    axes[i].set_xticks([])

plt.suptitle('Boxplots of Numerical Features (no removal of business-valid extremes)', fontsize=12)
plt.tight_layout()
plt.show()

## 6. Save Cleaned Data as Checkpoint

In [ ]:
# Save to CSV so N4 can start from the clean state without re-running earlier notebooks.
# index=False prevents pandas from adding an extra numeric index column.
df.to_csv('df_cleaned.csv', index=False)

print(f'✅ Cleaned data saved to  df_cleaned.csv')
print(f'   Shape: {df.shape}  ({df.shape[0]:,} rows × {df.shape[1]} columns)')
print(f'   Remaining columns: {df.columns.tolist()}')

---
## ✅ Cleaning Summary

| Step | Before | After |
|---|---|---|
| Columns | 18 | 14 |
| Rows | 10,000 | 10,000 |
| Missing values | 0 | 0 |
| Data leakage columns | 1 (`Complain`) | 0 |
| Identifier columns | 3 | 0 |

➡️ Continue to **N4_data_engineering** for feature scaling and encoding.
> Note: Class imbalance resampling (SMOTE) is handled in **N5**, after the train/test split, to avoid test-set contamination.